# Calculate confidence interval

This notebook generates the confidence interval for the plot in [stable_gene_relationships.ipynb](stable_gene_relationships.ipynb) notebook. Since this confidence interval is based on boostraping it takes a while to run so we wanted this in a separate notebook.

Existing ci calculation either assumes normality or uses bootstrapping, but since we need to make adjustments to normalize our results we cannot use the out of the box bootstrapping
https://stackoverflow.com/questions/46125182/is-seaborn-confidence-interval-computed-correctly

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import random
import scipy
import pandas as pd
import numpy as np
import textwrap
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.multitest import multipletests
from scripts import utils, paths, gene_relationships, annotations

random.seed(1)

/home/alexandra/anaconda3/envs/core_acc/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [2]:
# User params
offset_to_bin = 10

use_operon = True
sum_increment_to_use = 1

n_boot = 50

ci = 0.95

### Import gene ids


In [3]:
# Import correlation matrix to get gene ids
pao1_corr_filename = paths.PAO1_CORR_LOG_SPELL
pa14_corr_filename = paths.PA14_CORR_LOG_SPELL

pao1_corr = pd.read_csv(pao1_corr_filename, sep="\t", index_col=0, header=0)
pa14_corr = pd.read_csv(pa14_corr_filename, sep="\t", index_col=0, header=0)

In [4]:
# Make a dataframe with gene ids
pao1_membership = pd.DataFrame(data=[], index=pao1_corr.index)
print(pao1_membership.shape)
pao1_membership.head()

(5563, 0)


""
PA0001
PA0002
PA0003
PA0004
PA0005


In [5]:
pa14_membership = pd.DataFrame(data=[], index=pa14_corr.index)
print(pa14_membership.shape)
pa14_membership.head()

(5891, 0)


""
PA14_55610
PA14_55600
PA14_55590
PA14_55580
PA14_55570


### Import and format operon data

In [6]:
pao1_operon_filename = paths.PAO1_OPERON
pa14_operon_filename = paths.PA14_OPERON

In [7]:
pao1_operon = annotations.load_format_operons(pao1_operon_filename)
pa14_operon = annotations.load_format_operons(pa14_operon_filename)

In [8]:
if use_operon:
    pao1_operon_expression_to_use = pao1_operon
    pa14_operon_expression_to_use = pa14_operon
else:
    pao1_operon_expression_to_use = None
    pa14_operon_expression_to_use = None

### Map core/accessory labels to genes

In [9]:
# Read in expression data
pao1_expression_filename = paths.PAO1_COMPENDIUM
pa14_expression_filename = paths.PA14_COMPENDIUM

In [10]:
pao1_annot_filename = paths.GENE_PAO1_ANNOT
pa14_annot_filename = paths.GENE_PA14_ANNOT

In [11]:
(
    pao1_arr,
    pa14_arr,
    pao1_core,
    pao1_acc,
    pa14_core,
    pa14_acc,
) = annotations.map_core_acc_annot(
    pao1_membership,
    pa14_membership,
    pao1_expression_filename,
    pa14_expression_filename,
    pao1_annot_filename,
    pa14_annot_filename,
)

Number of PAO1 core genes: 5366
Number of PA14 core genes: 5363
Number of PAO1 core genes in my dataset: 5361
Number of PA14 core genes in my dataset: 5361
Number of PAO1-specific genes: 202
Number of PA14-specific genes: 530


## Find relationships using expression distance

In [12]:
# Correlation matrix files
pao1_corr_filename = paths.PAO1_CORR_LOG_SPELL
pa14_corr_filename = paths.PA14_CORR_LOG_SPELL

In [13]:
# Load correlation data
pao1_corr = pd.read_csv(pao1_corr_filename, sep="\t", index_col=0, header=0)
pa14_corr = pd.read_csv(pa14_corr_filename, sep="\t", index_col=0, header=0)

In [14]:
# Load transcriptional similarity df
# These are the subset of genes that we will consider
pao1_similarity_scores_filename = (
    "../3_core_core_analysis/pao1_core_similarity_associations_final_spell.tsv"
)
pa14_similarity_scores_filename = (
    "../3_core_core_analysis/pa14_core_similarity_associations_final_spell.tsv"
)

pao1_similarity_scores = pd.read_csv(
    pao1_similarity_scores_filename, sep="\t", header=0, index_col=0
)
pa14_similarity_scores = pd.read_csv(
    pa14_similarity_scores_filename, sep="\t", header=0, index_col=0
)

In [15]:
# Get most and least stable core genes
pao1_most_stable_genes = list(
    pao1_similarity_scores[pao1_similarity_scores["label"] == "most stable"].index
)
pao1_least_stable_genes = list(
    pao1_similarity_scores[pao1_similarity_scores["label"] == "least stable"].index
)

pa14_most_stable_genes = list(
    pa14_similarity_scores[pa14_similarity_scores["label"] == "most stable"].index
)
pa14_least_stable_genes = list(
    pa14_similarity_scores[pa14_similarity_scores["label"] == "least stable"].index
)

In [16]:
%%time
expression_dist_counts_pao1_most_ci = (
    gene_relationships.get_CI_expression_relationships(
        n_boot,
        pao1_corr,
        pao1_most_stable_genes,
        pao1_arr,
        offset_to_bin,
        pao1_operon_expression_to_use,
        sum_increment_to_use,
    )
)

CPU times: user 1h 19min 32s, sys: 11.2 s, total: 1h 19min 43s
Wall time: 1h 19min 26s


In [17]:
%%time
expression_dist_counts_pao1_least_ci = (
    gene_relationships.get_CI_expression_relationships(
        n_boot,
        pao1_corr,
        pao1_least_stable_genes,
        pao1_arr,
        offset_to_bin,
        pao1_operon_expression_to_use,
        sum_increment_to_use,
    )
)

CPU times: user 1h 20min 34s, sys: 12.2 s, total: 1h 20min 46s
Wall time: 1h 20min 28s


In [18]:
%%time
expression_dist_counts_pa14_most_ci = (
    gene_relationships.get_CI_expression_relationships(
        n_boot,
        pa14_corr,
        pa14_most_stable_genes,
        pa14_arr,
        offset_to_bin,
        pa14_operon_expression_to_use,
        sum_increment_to_use,
    )
)

CPU times: user 1h 24min 44s, sys: 16.8 s, total: 1h 25min 1s
Wall time: 1h 24min 33s


In [19]:
%%time
expression_dist_counts_pa14_least_ci = (
    gene_relationships.get_CI_expression_relationships(
        n_boot,
        pa14_corr,
        pa14_least_stable_genes,
        pa14_arr,
        offset_to_bin,
        pa14_operon_expression_to_use,
        sum_increment_to_use,
    )
)

CPU times: user 1h 25min 22s, sys: 16.5 s, total: 1h 25min 39s
Wall time: 1h 25min 12s


In [20]:
expression_dist_counts_pao1_most_ci

,offset,gene type,total_0,total_1,total_2,total_3,total_4,total_5,total_6,total_7,...,total_40,total_41,total_42,total_43,total_44,total_45,total_46,total_47,total_48,total_49
0,1,acc,3.000000,3.000000,6.000000,3.000000,3.000000,7.000000,4.000000,2.000000,...,6.000000,3.000000,3.000000,1.000000,5.000000,2.000000,4.000000,2.000000,6.000000,6.000000
1,2,acc,0.000000,1.000000,0.000000,1.000000,3.000000,3.000000,2.000000,1.000000,...,5.000000,2.000000,1.000000,2.000000,1.000000,2.000000,4.000000,2.000000,3.000000,1.000000
2,3,acc,4.000000,2.000000,2.000000,2.000000,0.000000,0.000000,2.000000,1.000000,...,1.000000,1.000000,1.000000,5.000000,7.000000,1.000000,2.000000,0.000000,3.000000,5.000000
3,4,acc,6.000000,3.000000,2.000000,1.000000,4.000000,2.000000,3.000000,1.000000,...,3.000000,2.000000,2.000000,6.000000,5.000000,6.000000,2.000000,3.000000,5.000000,2.000000
4,5,acc,3.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,2.000000,...,1.000000,1.000000,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,1.000000
5,6,acc,4.000000,2.000000,6.000000,7.000000,5.000000,4.000000,3.000000,4.000000,...,2.000000,5.000000,5.000000,10.000000,8.000000,4.000000,6.000000,5.000000,4.000000,2.000000
6,7,acc,3.000000,1.000000,2.000000,1.000000,1.000000,1.000000,2.000000,3.000000,...,2.000000,3.000000,1.000000,3.000000,2.000000,7.000000,4.000000,3.000000,4.000000,7.000000
7,8,acc,4.000000,3.000000,5.000000,3.000000,9.000000,4.000000,5.000000,9.000000,...,4.000000,2.000000,4.000000,6.000000,2.000000,3.000000,5.000000,4.000000,4.000000,9.000000
8,9,acc,2.000000,3.000000,2.000000,4.000000,2.000000,1.000000,2.000000,0.000000,...,1.000000,2.000000,4.000000,1.000000,3.000000,2.000000,0.000000,1.000000,3.000000,1.000000
9,10,acc,1.000000,1.000000,0.000000,2.000000,0.000000,1.000000,0.000000,1.000000,...,0.000000,2.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000


In [21]:
expression_dist_counts_pao1_least_ci

,offset,gene type,total_0,total_1,total_2,total_3,total_4,total_5,total_6,total_7,...,total_40,total_41,total_42,total_43,total_44,total_45,total_46,total_47,total_48,total_49
0,1,acc,16.000000,9.000000,15.000000,17.000000,13.000000,9.000000,7.000000,16.000000,...,15.000000,15.000000,14.000000,15.000000,19.000000,12.000000,20.000000,16.000000,9.000000,7.000000
1,2,acc,16.000000,15.000000,14.000000,11.000000,13.000000,11.000000,11.000000,12.000000,...,16.000000,12.000000,15.000000,12.000000,20.000000,11.000000,18.000000,9.000000,11.000000,10.000000
2,3,acc,17.000000,11.000000,11.000000,20.000000,8.000000,10.000000,14.000000,13.000000,...,16.000000,13.000000,16.000000,15.000000,14.000000,12.000000,13.000000,11.000000,9.000000,10.000000
3,4,acc,12.000000,11.000000,12.000000,14.000000,11.000000,12.000000,12.000000,9.000000,...,13.000000,15.000000,7.000000,17.000000,14.000000,10.000000,12.000000,16.000000,13.000000,11.000000
4,5,acc,13.000000,14.000000,11.000000,18.000000,13.000000,7.000000,16.000000,12.000000,...,22.000000,12.000000,15.000000,13.000000,16.000000,8.000000,15.000000,15.000000,9.000000,16.000000
5,6,acc,15.000000,15.000000,15.000000,15.000000,14.000000,10.000000,14.000000,12.000000,...,18.000000,17.000000,11.000000,17.000000,20.000000,12.000000,16.000000,13.000000,11.000000,12.000000
6,7,acc,12.000000,14.000000,11.000000,14.000000,15.000000,10.000000,15.000000,9.000000,...,16.000000,15.000000,8.000000,20.000000,9.000000,8.000000,17.000000,16.000000,11.000000,11.000000
7,8,acc,14.000000,13.000000,16.000000,19.000000,10.000000,15.000000,10.000000,12.000000,...,14.000000,15.000000,16.000000,16.000000,14.000000,11.000000,12.000000,11.000000,9.000000,12.000000
8,9,acc,9.000000,10.000000,8.000000,12.000000,8.000000,11.000000,14.000000,13.000000,...,11.000000,9.000000,11.000000,14.000000,10.000000,10.000000,12.000000,9.000000,9.000000,9.000000
9,10,acc,20.000000,16.000000,20.000000,14.000000,12.000000,13.000000,17.000000,21.000000,...,19.000000,22.000000,14.000000,22.000000,21.000000,21.000000,21.000000,16.000000,16.000000,13.000000


## Calculate percentages
Here we are taking the number of co-expressed core or accessory genes and normalizing by the number of most or least stable genes to get a percentage

In [22]:
# Get only columns with counts from bootstrapping
sampling_cols = [
    col for col in expression_dist_counts_pao1_most_ci.columns if "total" in col
]

In [23]:
pao1_acc_most_ids = expression_dist_counts_pao1_most_ci.loc[
    expression_dist_counts_pao1_most_ci["gene type"] == "acc"
].index
pao1_core_most_ids = expression_dist_counts_pao1_most_ci.loc[
    expression_dist_counts_pao1_most_ci["gene type"] == "core"
].index

pao1_acc_least_ids = expression_dist_counts_pao1_least_ci.loc[
    expression_dist_counts_pao1_least_ci["gene type"] == "acc"
].index
pao1_core_least_ids = expression_dist_counts_pao1_least_ci.loc[
    expression_dist_counts_pao1_least_ci["gene type"] == "core"
].index

In [24]:
pa14_acc_most_ids = expression_dist_counts_pa14_most_ci.loc[
    expression_dist_counts_pa14_most_ci["gene type"] == "acc"
].index
pa14_core_most_ids = expression_dist_counts_pa14_most_ci.loc[
    expression_dist_counts_pa14_most_ci["gene type"] == "core"
].index

pa14_acc_least_ids = expression_dist_counts_pa14_least_ci.loc[
    expression_dist_counts_pa14_least_ci["gene type"] == "acc"
].index
pa14_core_least_ids = expression_dist_counts_pa14_least_ci.loc[
    expression_dist_counts_pa14_least_ci["gene type"] == "core"
].index

In [25]:
# Most stable PAO1
expression_dist_counts_pao1_most_ci.loc[pao1_acc_most_ids, sampling_cols] /= len(
    pao1_most_stable_genes
)
expression_dist_counts_pao1_most_ci.loc[pao1_core_most_ids, sampling_cols] /= len(
    pao1_most_stable_genes
)

In [26]:
# Least stable PAO1
expression_dist_counts_pao1_least_ci.loc[pao1_acc_least_ids, sampling_cols] /= len(
    pao1_least_stable_genes
)
expression_dist_counts_pao1_least_ci.loc[pao1_core_least_ids, sampling_cols] /= len(
    pao1_least_stable_genes
)

In [27]:
# Most stable PA14
expression_dist_counts_pa14_most_ci.loc[pa14_acc_most_ids, sampling_cols] /= len(
    pa14_most_stable_genes
)
expression_dist_counts_pa14_most_ci.loc[pa14_core_most_ids, sampling_cols] /= len(
    pa14_most_stable_genes
)

In [28]:
# Least stable PA14
expression_dist_counts_pa14_least_ci.loc[pa14_acc_least_ids, sampling_cols] /= len(
    pa14_least_stable_genes
)
expression_dist_counts_pa14_least_ci.loc[pa14_core_least_ids, sampling_cols] /= len(
    pa14_least_stable_genes
)

In [29]:
expression_dist_counts_pao1_most_ci

,offset,gene type,total_0,total_1,total_2,total_3,total_4,total_5,total_6,total_7,...,total_40,total_41,total_42,total_43,total_44,total_45,total_46,total_47,total_48,total_49
0,1,acc,0.011236,0.011236,0.022472,0.011236,0.011236,0.026217,0.014981,0.007491,...,0.022472,0.011236,0.011236,0.003745,0.018727,0.007491,0.014981,0.007491,0.022472,0.022472
1,2,acc,0.000000,0.003745,0.000000,0.003745,0.011236,0.011236,0.007491,0.003745,...,0.018727,0.007491,0.003745,0.007491,0.003745,0.007491,0.014981,0.007491,0.011236,0.003745
2,3,acc,0.014981,0.007491,0.007491,0.007491,0.000000,0.000000,0.007491,0.003745,...,0.003745,0.003745,0.003745,0.018727,0.026217,0.003745,0.007491,0.000000,0.011236,0.018727
3,4,acc,0.022472,0.011236,0.007491,0.003745,0.014981,0.007491,0.011236,0.003745,...,0.011236,0.007491,0.007491,0.022472,0.018727,0.022472,0.007491,0.011236,0.018727,0.007491
4,5,acc,0.011236,0.000000,0.003745,0.003745,0.000000,0.000000,0.000000,0.007491,...,0.003745,0.003745,0.000000,0.000000,0.003745,0.007491,0.000000,0.000000,0.000000,0.003745
5,6,acc,0.014981,0.007491,0.022472,0.026217,0.018727,0.014981,0.011236,0.014981,...,0.007491,0.018727,0.018727,0.037453,0.029963,0.014981,0.022472,0.018727,0.014981,0.007491
6,7,acc,0.011236,0.003745,0.007491,0.003745,0.003745,0.003745,0.007491,0.011236,...,0.007491,0.011236,0.003745,0.011236,0.007491,0.026217,0.014981,0.011236,0.014981,0.026217
7,8,acc,0.014981,0.011236,0.018727,0.011236,0.033708,0.014981,0.018727,0.033708,...,0.014981,0.007491,0.014981,0.022472,0.007491,0.011236,0.018727,0.014981,0.014981,0.033708
8,9,acc,0.007491,0.011236,0.007491,0.014981,0.007491,0.003745,0.007491,0.000000,...,0.003745,0.007491,0.014981,0.003745,0.011236,0.007491,0.000000,0.003745,0.011236,0.003745
9,10,acc,0.003745,0.003745,0.000000,0.007491,0.000000,0.003745,0.000000,0.003745,...,0.000000,0.007491,0.000000,0.000000,0.003745,0.000000,0.003745,0.000000,0.000000,0.000000


In [30]:
expression_dist_counts_pao1_least_ci

,offset,gene type,total_0,total_1,total_2,total_3,total_4,total_5,total_6,total_7,...,total_40,total_41,total_42,total_43,total_44,total_45,total_46,total_47,total_48,total_49
0,1,acc,0.059925,0.033708,0.056180,0.063670,0.048689,0.033708,0.026217,0.059925,...,0.056180,0.056180,0.052434,0.056180,0.071161,0.044944,0.074906,0.059925,0.033708,0.026217
1,2,acc,0.059925,0.056180,0.052434,0.041199,0.048689,0.041199,0.041199,0.044944,...,0.059925,0.044944,0.056180,0.044944,0.074906,0.041199,0.067416,0.033708,0.041199,0.037453
2,3,acc,0.063670,0.041199,0.041199,0.074906,0.029963,0.037453,0.052434,0.048689,...,0.059925,0.048689,0.059925,0.056180,0.052434,0.044944,0.048689,0.041199,0.033708,0.037453
3,4,acc,0.044944,0.041199,0.044944,0.052434,0.041199,0.044944,0.044944,0.033708,...,0.048689,0.056180,0.026217,0.063670,0.052434,0.037453,0.044944,0.059925,0.048689,0.041199
4,5,acc,0.048689,0.052434,0.041199,0.067416,0.048689,0.026217,0.059925,0.044944,...,0.082397,0.044944,0.056180,0.048689,0.059925,0.029963,0.056180,0.056180,0.033708,0.059925
5,6,acc,0.056180,0.056180,0.056180,0.056180,0.052434,0.037453,0.052434,0.044944,...,0.067416,0.063670,0.041199,0.063670,0.074906,0.044944,0.059925,0.048689,0.041199,0.044944
6,7,acc,0.044944,0.052434,0.041199,0.052434,0.056180,0.037453,0.056180,0.033708,...,0.059925,0.056180,0.029963,0.074906,0.033708,0.029963,0.063670,0.059925,0.041199,0.041199
7,8,acc,0.052434,0.048689,0.059925,0.071161,0.037453,0.056180,0.037453,0.044944,...,0.052434,0.056180,0.059925,0.059925,0.052434,0.041199,0.044944,0.041199,0.033708,0.044944
8,9,acc,0.033708,0.037453,0.029963,0.044944,0.029963,0.041199,0.052434,0.048689,...,0.041199,0.033708,0.041199,0.052434,0.037453,0.037453,0.044944,0.033708,0.033708,0.033708
9,10,acc,0.074906,0.059925,0.074906,0.052434,0.044944,0.048689,0.063670,0.078652,...,0.071161,0.082397,0.052434,0.082397,0.078652,0.078652,0.078652,0.059925,0.059925,0.048689


## Normalize by base percentage
Here we want to normalize the percentage of co-expressed genes with the percent of accessory or core genes in the genome.

In [31]:
# Baseline/expected proportions for PAO1
pao1_total = len(pao1_core) + len(pao1_acc)
pao1_acc_expected = len(pao1_acc) / pao1_total
pao1_core_expected = len(pao1_core) / pao1_total
print("total pao1 genes", pao1_total)
print("pao1 acc baseline", pao1_acc_expected)
print("pao1 core baseline", pao1_core_expected)

total pao1 genes 5563
pao1 acc baseline 0.036311342800647135
pao1 core baseline 0.9636886571993528


In [32]:
# Baseline/expected proportions for PA14
pa14_total = len(pa14_core) + len(pa14_acc)
pa14_acc_expected = len(pa14_acc) / pa14_total
pa14_core_expected = len(pa14_core) / pa14_total
print("total pa14 genes", pa14_total)
print("pa14 acc baseline", pa14_acc_expected)
print("pa14 core baseline", pa14_core_expected)

total pa14 genes 5891
pa14 acc baseline 0.08996774741130538
pa14 core baseline 0.9100322525886946


In [33]:
# Most stable PAO1
expression_dist_counts_pao1_most_ci.loc[
    pao1_acc_most_ids, sampling_cols
] /= pao1_acc_expected
expression_dist_counts_pao1_most_ci.loc[
    pao1_core_most_ids, sampling_cols
] /= pao1_core_expected

In [34]:
# Least stable PAO1
expression_dist_counts_pao1_least_ci.loc[
    pao1_acc_least_ids, sampling_cols
] /= pao1_acc_expected
expression_dist_counts_pao1_least_ci.loc[
    pao1_core_least_ids, sampling_cols
] /= pao1_core_expected

In [35]:
# Most stable PA14
expression_dist_counts_pa14_most_ci.loc[
    pa14_acc_most_ids, sampling_cols
] /= pa14_acc_expected
expression_dist_counts_pa14_most_ci.loc[
    pa14_core_most_ids, sampling_cols
] /= pa14_core_expected

In [36]:
# Least stable PA14
expression_dist_counts_pa14_least_ci.loc[
    pa14_acc_least_ids, sampling_cols
] /= pa14_acc_expected
expression_dist_counts_pa14_least_ci.loc[
    pa14_core_least_ids, sampling_cols
] /= pa14_core_expected

In [37]:
expression_dist_counts_pao1_most_ci

,offset,gene type,total_0,total_1,total_2,total_3,total_4,total_5,total_6,total_7,...,total_40,total_41,total_42,total_43,total_44,total_45,total_46,total_47,total_48,total_49
0,1,acc,0.309434,0.309434,0.618868,0.309434,0.309434,0.722012,0.412578,0.206289,...,0.618868,0.309434,0.309434,0.103145,0.515723,0.206289,0.412578,0.206289,0.618868,0.618868
1,2,acc,0.000000,0.103145,0.000000,0.103145,0.309434,0.309434,0.206289,0.103145,...,0.515723,0.206289,0.103145,0.206289,0.103145,0.206289,0.412578,0.206289,0.309434,0.103145
2,3,acc,0.412578,0.206289,0.206289,0.206289,0.000000,0.000000,0.206289,0.103145,...,0.103145,0.103145,0.103145,0.515723,0.722012,0.103145,0.206289,0.000000,0.309434,0.515723
3,4,acc,0.618868,0.309434,0.206289,0.103145,0.412578,0.206289,0.309434,0.103145,...,0.309434,0.206289,0.206289,0.618868,0.515723,0.618868,0.206289,0.309434,0.515723,0.206289
4,5,acc,0.309434,0.000000,0.103145,0.103145,0.000000,0.000000,0.000000,0.206289,...,0.103145,0.103145,0.000000,0.000000,0.103145,0.206289,0.000000,0.000000,0.000000,0.103145
5,6,acc,0.412578,0.206289,0.618868,0.722012,0.515723,0.412578,0.309434,0.412578,...,0.206289,0.515723,0.515723,1.031446,0.825157,0.412578,0.618868,0.515723,0.412578,0.206289
6,7,acc,0.309434,0.103145,0.206289,0.103145,0.103145,0.103145,0.206289,0.309434,...,0.206289,0.309434,0.103145,0.309434,0.206289,0.722012,0.412578,0.309434,0.412578,0.722012
7,8,acc,0.412578,0.309434,0.515723,0.309434,0.928301,0.412578,0.515723,0.928301,...,0.412578,0.206289,0.412578,0.618868,0.206289,0.309434,0.515723,0.412578,0.412578,0.928301
8,9,acc,0.206289,0.309434,0.206289,0.412578,0.206289,0.103145,0.206289,0.000000,...,0.103145,0.206289,0.412578,0.103145,0.309434,0.206289,0.000000,0.103145,0.309434,0.103145
9,10,acc,0.103145,0.103145,0.000000,0.206289,0.000000,0.103145,0.000000,0.103145,...,0.000000,0.206289,0.000000,0.000000,0.103145,0.000000,0.103145,0.000000,0.000000,0.000000


In [38]:
expression_dist_counts_pao1_least_ci

,offset,gene type,total_0,total_1,total_2,total_3,total_4,total_5,total_6,total_7,...,total_40,total_41,total_42,total_43,total_44,total_45,total_46,total_47,total_48,total_49
0,1,acc,1.650313,0.928301,1.547169,1.753458,1.340880,0.928301,0.722012,1.650313,...,1.547169,1.547169,1.444024,1.547169,1.959747,1.237735,2.062892,1.650313,0.928301,0.722012
1,2,acc,1.650313,1.547169,1.444024,1.134590,1.340880,1.134590,1.134590,1.237735,...,1.650313,1.237735,1.547169,1.237735,2.062892,1.134590,1.856603,0.928301,1.134590,1.031446
2,3,acc,1.753458,1.134590,1.134590,2.062892,0.825157,1.031446,1.444024,1.340880,...,1.650313,1.340880,1.650313,1.547169,1.444024,1.237735,1.340880,1.134590,0.928301,1.031446
3,4,acc,1.237735,1.134590,1.237735,1.444024,1.134590,1.237735,1.237735,0.928301,...,1.340880,1.547169,0.722012,1.753458,1.444024,1.031446,1.237735,1.650313,1.340880,1.134590
4,5,acc,1.340880,1.444024,1.134590,1.856603,1.340880,0.722012,1.650313,1.237735,...,2.269181,1.237735,1.547169,1.340880,1.650313,0.825157,1.547169,1.547169,0.928301,1.650313
5,6,acc,1.547169,1.547169,1.547169,1.547169,1.444024,1.031446,1.444024,1.237735,...,1.856603,1.753458,1.134590,1.753458,2.062892,1.237735,1.650313,1.340880,1.134590,1.237735
6,7,acc,1.237735,1.444024,1.134590,1.444024,1.547169,1.031446,1.547169,0.928301,...,1.650313,1.547169,0.825157,2.062892,0.928301,0.825157,1.753458,1.650313,1.134590,1.134590
7,8,acc,1.444024,1.340880,1.650313,1.959747,1.031446,1.547169,1.031446,1.237735,...,1.444024,1.547169,1.650313,1.650313,1.444024,1.134590,1.237735,1.134590,0.928301,1.237735
8,9,acc,0.928301,1.031446,0.825157,1.237735,0.825157,1.134590,1.444024,1.340880,...,1.134590,0.928301,1.134590,1.444024,1.031446,1.031446,1.237735,0.928301,0.928301,0.928301
9,10,acc,2.062892,1.650313,2.062892,1.444024,1.237735,1.340880,1.753458,2.166036,...,1.959747,2.269181,1.444024,2.269181,2.166036,2.166036,2.166036,1.650313,1.650313,1.340880


## Get quantiles

In [39]:
alpha = 1 - ci
lower = alpha / 2
upper = 1 - (alpha / 2)

In [40]:
pao1_most_ci_ranges = expression_dist_counts_pao1_most_ci.quantile(
    [lower, upper], axis=1
)
pao1_least_ci_ranges = expression_dist_counts_pao1_least_ci.quantile(
    [lower, upper], axis=1
)

In [41]:
pa14_most_ci_ranges = expression_dist_counts_pa14_most_ci.quantile(
    [lower, upper], axis=1
)
pa14_least_ci_ranges = expression_dist_counts_pa14_least_ci.quantile(
    [lower, upper], axis=1
)

## Format

Merge with starting df with corr

In [42]:
pao1_most_ci = expression_dist_counts_pao1_most_ci.merge(
    pao1_most_ci_ranges.T, left_index=True, right_index=True
).drop(sampling_cols, axis=1)
pao1_least_ci = expression_dist_counts_pao1_least_ci.merge(
    pao1_least_ci_ranges.T, left_index=True, right_index=True
).drop(sampling_cols, axis=1)

pao1_most_ci.columns = ["offset", "gene type", "ymin", "ymax"]
pao1_least_ci.columns = ["offset", "gene type", "ymin", "ymax"]

In [43]:
pa14_most_ci = expression_dist_counts_pa14_most_ci.merge(
    pa14_most_ci_ranges.T, left_index=True, right_index=True
).drop(sampling_cols, axis=1)
pa14_least_ci = expression_dist_counts_pa14_least_ci.merge(
    pa14_least_ci_ranges.T, left_index=True, right_index=True
).drop(sampling_cols, axis=1)

pa14_most_ci.columns = ["offset", "gene type", "ymin", "ymax"]
pa14_least_ci.columns = ["offset", "gene type", "ymin", "ymax"]

In [44]:
pao1_most_ci.head()

,offset,gene type,ymin,ymax
0,1,acc,0.103145,0.722012
1,2,acc,0.000000,0.492515
2,3,acc,0.000000,0.515723
3,4,acc,0.000000,0.618868
4,5,acc,0.000000,0.286226


In [45]:
pa14_most_ci.head()

,offset,gene type,ymin,ymax
0,1,acc,0.0,0.166518
1,2,acc,0.0,0.208148
2,3,acc,0.0,0.166518
3,4,acc,0.0,0.323670
4,5,acc,0.0,0.166518


In [46]:
# Save
pao1_most_ci.to_csv("pao1_most_ci.tsv", sep="\t")
pao1_least_ci.to_csv("pao1_least_ci.tsv", sep="\t")

pa14_most_ci.to_csv("pa14_most_ci.tsv", sep="\t")
pa14_least_ci.to_csv("pa14_least_ci.tsv", sep="\t")